In [1]:
import numpy as np
import pandas as pd
import torch
import esm
import scipy
from sklearn.model_selection import GridSearchCV, train_test_split
from numpy import asarray
from numpy import savez_compressed

In [2]:
PDB_csv_path = "/home/PC/Desktop/Protein_language_model/embeddings/PDB_LIST.csv"
PDB_name_df = pd.read_csv(PDB_csv_path)
PDBS = PDB_name_df['PDB_ID'].tolist()

In [3]:

def exctracting_embeddings_esm2(pdb):
    mutations2= [] 
    Xs2 = []
    for header2, _seq2 in esm.data.read_fasta(FASTA_PATH2):
        scaled_effect2 = header2.split('|')[-1]
        mutations2.append(scaled_effect2)
        fn = f'{EMB_PATH2}/{header2[1:]}.pt'
        embs2 = torch.load(fn)
        Xs2.append(embs2['representations'][33])
    Xs2 = torch.stack(Xs2, dim=0).numpy()
    
    return Xs2
    

    
    
    
def exctracting_embeddings_esm2_wild(pdb):
    df = pd.read_csv(csv_path)
    ys = df['bind'].tolist()
    no_of_mutations = len(ys)
    mutations2_w= [] 
    Xs2_w = []
    for header2, _seq2 in esm.data.read_fasta(FASTA_PATH2_w):
        scaled_effect2_w = header2.split('|')[-1]
        mutations2_w.append(scaled_effect2_w)
        fn = f'{EMB_PATH2_w}/{header2[1:]}.pt'
        embs2 = torch.load(fn)
        Xs2_w.append(embs2['representations'][33])
    Xs2_w = torch.stack(Xs2_w, dim=0).numpy()
    Xs2_w=np.tile(Xs2_w, (no_of_mutations, 1, 1))
    
    return Xs2_w
    
    
    
    
def exctracting_embeddings_esm2_bind(pdb):
    df = pd.read_csv(csv_path)
    ys = df['bind'].tolist()
    no_of_mutations = len(ys)
    mutations2_b= [] 
    Xs2_b = []
    for header2, _seq2 in esm.data.read_fasta(FASTA_PATH2_b):
        scaled_effect2_b = header2.split('|')[-1]
        mutations2_b.append(scaled_effect2_b)
        fn = f'{EMB_PATH2_b}/{header2[1:]}.pt'
        embs2 = torch.load(fn)
        Xs2_b.append(embs2['representations'][33])
    Xs2_b = torch.stack(Xs2_b, dim=0).numpy()
    Xs2_b=np.tile(Xs2_b, (no_of_mutations, 1, 1))
    
    return Xs2_b, ys


def exctracting_embeddings_1f(pdb):
    df = pd.read_csv(csv_path)
    ys = df['bind'].tolist()
    no_of_mutations = len(ys)
    temp= np.load(inverse_path)
    inverse= temp['data']
    

    average_mean_embedding = np.mean(inverse, axis=0)
    average_mean_embedding.shape
    inverse_mean_reshape = average_mean_embedding.reshape([1, 512])
    inverse_mean_reshape.shape
    inverse_mean_228=np.tile(inverse_mean_reshape, (no_of_mutations, 1))
    

    
    return inverse_mean_228, ys

In [4]:
ddg_values = []
embeddings = []
for pdb in PDBS:
    FASTA_PATH = "/home/PC/Desktop/Protein_language_model/embeddings/{}.fasta".format(pdb)
    EMB_PATH = "/home/PC/Desktop/Protein_language_model/embeddings/{}_1V".format(pdb)
    FASTA_PATH2 = "//home/PC/Desktop/Protein_language_model/embeddings/{}.fasta".format(pdb)
    EMB_PATH2 = "/home/PC/Desktop/Protein_language_model/embeddings/{}_esm2".format(pdb)
    FASTA_PATH_w = "/home/PC/Desktop/Protein_language_model/embeddings/{}_wild.fasta".format(pdb)
    EMB_PATH_w = "/home/PC/Desktop/Protein_language_model/embeddings/{}_1V_wild".format(pdb)
    FASTA_PATH2_w = "/home/PC/Desktop/Protein_language_model/embeddings/{}_wild.fasta".format(pdb)
    EMB_PATH2_w = "/home/PC/Desktop/Protein_language_model/embeddings/{}_esm2_wild".format(pdb)
    FASTA_PATH_b = "/home/PC/Desktop/Protein_language_model/embeddings/{}_partner.fasta".format(pdb)
    EMB_PATH_b = "/home/PC/Desktop/Protein_language_model/embeddings/{}_1V_partner".format(pdb)
    FASTA_PATH2_b = "/home/PC/Desktop/Protein_language_model/embeddings/{}_partner.fasta".format(pdb)
    EMB_PATH2_b = "/home/PC/Desktop/Protein_language_model/embeddings/{}_esm2_partner".format(pdb)
    inverse_path = '/home/PC/Desktop/Protein_language_model/embeddings/inverse_{}.npz'.format(pdb)
    csv_path = "/home/PC/Desktop/Protein_language_model/embeddings/{}.csv".format(pdb)
    Xs2= exctracting_embeddings_esm2(pdb)
    Xs2_w= exctracting_embeddings_esm2_wild(pdb)
    Xs2_b, ys=exctracting_embeddings_esm2_bind(pdb)
    inverse, ys=exctracting_embeddings_1f(pdb)
    mutant_and_partner_together_esm2 = np.concatenate([Xs2_b, Xs2], axis =1)
    
    wild_type_and_partner_together_esm2 = np.concatenate([Xs2_b, Xs2_w], axis =1)
    mutant_and_partner_together_esm2_mean=np.mean(mutant_and_partner_together_esm2, axis=1)
    wild_type_and_partner_together_esm2_mean=np.mean(wild_type_and_partner_together_esm2, axis=1)    
    ddg_1v = np.subtract(mutant_and_partner_together_esm2_mean, wild_type_and_partner_together_esm2_mean) 
    
    ddg_esm2_with_inverse = np.concatenate([ddg_1v, inverse], axis =1)
    embeddings.append(ddg_esm2_with_inverse)
    ddg_values.append(ys)

In [5]:
flattened_list = [item for sublist in ddg_values for item in sublist]

In [6]:
extracted_array = embeddings[0]

In [7]:
train_size = 0.8
Xs_train, Xs_test, ys_train, ys_test= train_test_split(extracted_array, flattened_list, train_size=train_size, random_state=42)
Xs_train.shape, Xs_test.shape, len(ys_train), len(ys_test)

((152, 1792), (38, 1792), 152, 38)

In [8]:
np.savez('train.npz', data=Xs_train, label=ys_train)

In [9]:
np.savez('test.npz', data=Xs_test, label=ys_test)